In [1]:
#define sequence

seq = 'AGGACTATAAATAGCAGCCACCTCTCCCTGGCAGACAGGGACCCGCAGCTCAGCTACAGCACAGATCAGCACCATGCCTCTGCTGTGGCTGAGAGGCTTCCTGCTGGCCTCTTGCTGGATTATTGTGAGATCTTCTCCTACACCTGGCTCTGAGGGCCACGGCTCTGCCCCTGACTGCCCTTCTTGCGCCCTGGCCACACTGCCTAAGGACGGCCCTAACTCTCAGCCTGAGATGGTGGAGGCCGTGAAGAAGCACATTCTGAACATGCTGCACCTGAAGAAGAGACCTGACGTGACACAGCCTGTGCCTAAGGCCGCCCTGCTGAACGCCATTAGAAAGCTGCACGTGGGCAAGGTGGGCGAGAACGGCTACGTGGAGATTGAGGACGACATTGGCAGAAGAGCCGAGATGAACGAGCTGATGGAGCAGACATCTGAGATTATTACATTCGCCGAGTCTGGCACAGCCAGAAAGACACTGCACTTCGAGATTTCTAAGGAGGGCTCTGACCTGTCTGTGGTGGAGAGAGCCGAGGTGTGGCTGTTCCTGAAGGTGCCTAAGGCCAACAGAACAAGAACAAAGGTGACAATTAGACTGTTCCAGCAGCAGAAGCACCCTCAGGGCTCTCTGGACACAGGCGACGAGGCCGAGGAGATGGGCCTGAAGGGCGAGAGATCTGAGCTGCTGCTGTCTGAGAAGGTGGTGGACGCCAGAAAGTCTACATGGCACATTTTCCCTGTGTCTTCTTCTATTCAGAGACTGCTGGACCAGGGCAAGTCTTCTCTGGACGTGAGAATTGCCTGCGAGCAGTGCCAGGAGTCTGGCGCCTCTCTGGTGCTGCTGGGCAAGAAGAAGAAGAAGGAGGTGGACGGCGACGGCAAGAAGAAGGACGGCTCTGACGGCGGCCTGGAGGAGGAGAAGGAGCAGTCTCACAGACCTTTCCTGATGCTGCAGGCCAGACAGTCTGAGGACCACCCTCACAGAAGAAGAAGAAGAGGCCTGGAGTGCGACGGCAAGGTGAACATTTGCTGCAAGAAGCAGTTCTTCGTGTCTTTCAAGGACATTGGCTGGAACGACTGGATTATTGCCCCTTCTGGCTACCACGCCAACTACTGCGAGGGCGAGTGCCCTTCTCACATTGCCGGCACATCTGGCTCTTCTCTGTCTTTCCACTCTACAGTGATTAACCACTACAGAATGAGAGGCCACTCTCCTTTCGCCAACCTGAAGTCTTGCTGCGTGCCTACAAAGCTGAGACCTATGTCTATGCTGTACTACGACGACGGCCAGAACATTATTAAGAAGGACATTCAGAACATGATTGTGGAGGAGTGCGGCTGCTCTTAGGCTTCCTCTTCACTCTGCTCTCAGGAGACCTGGCTATGAGGCCCTCGGGGCAGGGATACAAAGTTAGTGAGGTCTATGTCCAGAGAAGCTGAGATATGGCATATAATAGGCATCTAATAAATGCTTAAGAGGTGGAA'

In [2]:
#generate analyses
import matplotlib.pyplot as plt
from src.seqview import basicView
from PIL import Image

fig,ax = basicView(seq)
plt.savefig('assets/Sequence.png')
plt.close()


from src.codon_comp import plotCodonCounts

fig,ax = plotCodonCounts(seq)
plt.savefig('assets/CodonCounts.png')
plt.close()

from src.frameshift import mapFrameshifts

fig,ax = mapFrameshifts(seq)
plt.savefig('assets/FS.png')
plt.close()

from src.repeats import reportNtRepeats

fig,ax = reportNtRepeats(seq)
plt.savefig('assets/repeats.png')
plt.close()

from src.terminators import plotTypeIITranscriptionTerminators, plotStemLoops

fig,ax = plotTypeIITranscriptionTerminators(seq)
plt.savefig('assets/typeIIterm.png')
plt.close()

fig,ax = plotStemLoops(seq)
plt.savefig('assets/typeIterm.png')
plt.close()

pil_sequence = Image.open('assets/Sequence.png')
pil_ccounts = Image.open('assets/CodonCounts.png')
pil_FS = Image.open('assets/FS.png')
pil_repeats = Image.open('assets/repeats.png')
pil_FS = Image.open('assets/typeIIterm.png')
pil_FS = Image.open('assets/typeIterm.png')

In [17]:
from dash import Dash, html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc
from PIL import Image

width='40px'
height='70px'

app=Dash(external_stylesheets=['assets/local_styles.css',dbc.themes.BOOTSTRAP])
app.title = "lintYR - an RNA lint tool"

app.layout = dbc.Container\
(className='custom-container',children=[
    dbc.Row(children=[
        dbc.Col(children=[html.H2('Title Row',style={'text-align':'center','font-style':'bold'})])
    ]),
    dbc.Row([
        dbc.Col([
            html.Img(src=Image.open('assets/Sequence.png'),style={'width':'100%'})
        ],width=6),
        dbc.Col([
            dbc.Row([
                dbc.Col([html.Img(src=Image.open('assets/traffic_light_green.png'),style={'width':width,'height':height})]),
                dbc.Col([html.Img(src=Image.open('assets/traffic_light_green.png'),style={'width':width,'height':height})]),
                dbc.Col([html.Img(src=Image.open('assets/traffic_light_green.png'),style={'width':width,'height':height})]),
                dbc.Col([html.Img(src=Image.open('assets/traffic_light_green.png'),style={'width':width,'height':height})]),
                dbc.Col([html.Img(src=Image.open('assets/traffic_light_green.png'),style={'width':width,'height':height})]),
            ],align='start',justify='around'),
            dbc.Row([
                dbc.Col([
                    dcc.Tabs(id='display_tabs',value='Codon Usage',parent_className ='custom-tabs',className='custom-tabs-container',children=[
                        dcc.Tab(label='Codon Usage', value='Codon Usage',className='custom-tab',selected_className='custom-tab--selected'),
                        dcc.Tab(label='Slippery Sites', value='FS',className='custom-tab',selected_className='custom-tab--selected'),
                        dcc.Tab(label='Type 1 Terminators', value='Type 1 Term',className='custom-tab',selected_className='custom-tab--selected'),
                        dcc.Tab(label='Type 2 Terminators', value='Type 2 Term',className='custom-tab',selected_className='custom-tab--selected'),
                        dcc.Tab(label='Repeats', value='Repeats',className='custom-tab',selected_className='custom-tab--selected'),
                     ]), 
                ]),
            ]),
            dbc.Row([html.Div(id='tabbed-graph')])
        ],width=6),
    ]),
])

@callback(Output('tabbed-graph', 'children'),
          Input('display_tabs', 'value'))
def render_content(tab):
    if tab == 'Codon Usage':
        return html.Div([
            html.Img(src=Image.open('assets/CodonCounts.png'),style={'width':'100%','padding-bottom':'24px'}),
            dcc.Markdown('''
                Codon Usage is important to ensure high expression levels from *in vitro* transcribed
                mRNAs. Optimised sequences predominantly or exclusively use one codon for each of the 20 amino acids.
            '''),
        ])
    elif tab == 'FS':
        return html.Div([
            html.Img(src=Image.open('assets/FS.png'),style={'width':'100%','padding-bottom':'24px'}),
            dcc.Markdown('''
                Slippery sites sites are nucleotide sequences where ribosomes have a high tendency to 
                slip into +1 reading frames, especially in *in vitro* transcribed RNAs 
                containing modified nucleotides such as N1-methyl-pseudouridine.
                
                This effect was originally described in [Mulroney *et al* 2024](https://doi.org/10.1038/s41586-023-06800-3)
            ''',link_target="_blank", style={'color':'#C9C9C9'}),
        ])
    elif tab == 'Type 1 Term':
        return html.Div([
            html.Img(src=Image.open('assets/typeIterm.png'),style={'width':'100%','padding-bottom':'24px'}),
            dcc.Markdown('''
                Repeat text to be completed
            ''',link_target="_blank",),
        ])
    elif tab == 'Type 2 Term':
        return html.Div([
            html.Img(src=Image.open('assets/typeIIterm.png'),style={'width':'100%','padding-bottom':'24px'}),
            dcc.Markdown('''
                Repeat text to be completed
            ''',link_target="_blank",),
        ])
    elif tab == 'Repeats':
        return html.Div([
            html.Img(src=Image.open('assets/repeats.png'),style={'width':'100%','padding-bottom':'24px'}),
            dcc.Markdown('''
                Repeat text to be completed
            ''',link_target="_blank",),
        ])

app.run(jupyter_mode='tab')

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>